In [9]:
import re
import math
import glob
import numpy as np
import pandas as pd 
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy.spatial import Voronoi, voronoi_plot_2d
import freud
from matplotlib import collections, colors, transforms
import networkx as nx

In [10]:
def get_checkpoints(dir):
    filelist = glob.glob(dir+"/positions_*.bin")
    g = lambda x: x.split("_")[-1].split(".")[0]
    checkpoints = [ int(g(l)) for l in filelist]
    checkpoints.sort(reverse = True)
    return checkpoints
    
def get_param_from_dir(dir):
    subdir = dir.split("/",5)[-1]
    ptype, paramstring = subdir.split("_",1)
    return ptype, re.findall(r"[-+]?\d*\.\d+|\d+\d?", paramstring)

def get_state():
    pos = np.fromfile(dir+"/{}_{}.bin".format('positions', cp))
    #pos = np.reshape(pos, (-1,3))
    #print(pos)
    #print(np.shape(pos))

    orient = np.fromfile(dir+"/{}_{}.bin".format('orientations', cp))
    orient = np.reshape(orient, (-1,5))[:,4]
    #print(np.shape(orient))

    box = np.fromfile(dir+"/{}_{}.bin".format('Box', cp))
    #box_xy = box[3:5]
    #print(box_xy)
    
    N_particles = len(orient)
    return N_particles, np.reshape(pos, (-1,3)), orient, box[3:5]
    

def rotation_matrix(theta):
    c, s = np.cos(theta), np.sin(theta) 
    R = np.array(((c, -s), (s, c)))

    return R
    
def get_orient(v, rot_mat):
    return rot_mat.dot(v)

def get_triangle_vertices_i(pos_i, orient_i):
    vertices = np.zeros((3,3))

    l2 = 0.5
    h2_small = np.sqrt(3)/6
    h2_large = 2*h2_small

    rotmat_i = rotation_matrix(orient_i)
    ax0 = np.array([[-l2, l2, 0], [-h2_small, -h2_small, h2_large]])

    ax_n = get_orient(ax0, rotmat_i).T

    vertices[0,0:2] = pos_i[0:2] + ax_n[0]
    vertices[1,0:2] = pos_i[0:2] + ax_n[1]
    vertices[2,0:2] = pos_i[0:2] + ax_n[2]

    return vertices

def get_number_of_patches(type):
    if(type=="6patch"):
        #six_patch
        return 6
    elif(type=="3asym"):
        #three_asymm
        return 3
    elif(type=="vo"):
        #veritce_opposite
        return 2
    elif(type=="vn"):
        #vertice_neighbour
        return 2
    elif(type=="2asym_c"):
        #two_asymm_center
        return 3
    elif(type=="mouse"):
        #mouse
        return 3
    else:
        print("Type not defined in function get_number_of_patches. Returned 0.")
        return 0

def get_patch_pos_i(vertices_i, type, delta):
    if(type=="6patch"):
        #six_patch
        patch_pos = np.zeros((6,3))
        patch_pos[0] = vertices_i[0] + (vertices_i[1]-vertices_i[0]) * delta
        patch_pos[1] = vertices_i[0] + (vertices_i[1]-vertices_i[0]) * (1 - delta)
        patch_pos[2] = vertices_i[1] + (vertices_i[2]-vertices_i[1]) * delta
        patch_pos[3] = vertices_i[1] + (vertices_i[2]-vertices_i[1]) * (1 - delta)
        patch_pos[4] = vertices_i[2] + (vertices_i[0]-vertices_i[2]) * delta
        patch_pos[5] = vertices_i[2] + (vertices_i[0]-vertices_i[2]) * (1 - delta)

    elif(type=="3asym"):
        #three_asymm
        patch_pos = np.zeros((3,3))
        patch_pos[1] = vertices_i[1] + (vertices_i[2]-vertices_i[1]) * delta
        patch_pos[2] = vertices_i[2] + (vertices_i[0]-vertices_i[2]) * delta
        patch_pos[0] = vertices_i[0] + (vertices_i[1]-vertices_i[0]) * delta

    elif(type=="vo"):
        #veritce_opposite
        patch_pos = np.zeros((2,3))
        patch_pos[0] = vertices_i[0]
        patch_pos[1] = vertices_i[1] + (vertices_i[2]-vertices_i[1]) * delta


    elif(type=="vn"):
        #vertice_neighbour
        patch_pos = np.zeros((2,3))
        patch_pos[0] = vertices_i[0]
        patch_pos[1] = vertices_i[0] + (vertices_i[1]-vertices_i[0]) * delta


    elif(type=="2asym_c"):
        #two_asymm_center
        patch_pos = np.zeros((3,2))
        patch_pos[0] = vertices_i[0] + (vertices_i[1]-vertices_i[0]) * 0.5
        patch_pos[1] = vertices_i[1] + (vertices_i[2]-vertices_i[1]) * delta
        patch_pos[2] = vertices_i[2] + (vertices_i[0]-vertices_i[2]) * delta

    elif(type=="mouse"):
        #mouse
        patch_pos = np.zeros((3,2))
        patch_pos[0] = vertices_i[0] + (vertices_i[1]-vertices_i[0]) * delta
        patch_pos[1] = vertices_i[1] + (vertices_i[2]-vertices_i[1]) * (1 - delta)
        patch_pos[2] = vertices_i[2] + (vertices_i[0]-vertices_i[2]) * delta
    else:
        print("Type not defined in function get_patch_pos_i. Returned 0.")
        return 0
    return patch_pos

def get_vertices_patches():
    vertices = np.zeros((N_particles,3,3))
    patches = np.zeros((N_particles,N_patches_i,3))
    for i in range(N_particles):
        vertices[i] = get_triangle_vertices_i(positions[i], orient[i])
        patches[i] = get_patch_pos_i(vertices[i], param[0], float(param[1][0]))
    return vertices, patches

def collect_and_transform_points_for_voronoi():
    vplot_vertices = np.copy(np.reshape(vertices,(N_particles*3,3)))
    vplot_vertices[:,:2]-=box_length/2

    vplot_points = np.copy(positions)
    vplot_points[:,:2]-= box_length/2
    vplot_points[:,2]=0
    vplot_points = np.append(vplot_points,vplot_vertices,axis=0)

    return vplot_points

def get_voronoi_neighbours(connections):
    neighbours_as_slices = np.ndarray((len(vplot_points)),dtype=object)
    list_idx = 0
    cell_idx = 0
    for i,con in enumerate(connections):
        if con[0] != cell_idx or i == len(connections)-1:
            neighbours_as_slices[cell_idx] = slice(list_idx,i,1)
            list_idx=i
            cell_idx+=1

    idx_first_neighbours = np.ndarray(len(vplot_points), dtype=object)
    for i,slc in enumerate(neighbours_as_slices):
        idx_first_neighbours[i] = connections[slc,1:].flatten()

    idx_second_neighbours = np.copy(idx_first_neighbours)
    for i,neighs in enumerate(idx_first_neighbours):
        for idx in neighs:
            idx_second_neighbours[i] = np.unique(np.append(idx_second_neighbours[i],idx_first_neighbours[idx]))
            idx_second_neighbours[i] = idx_second_neighbours[i][idx_second_neighbours[i] != i]
    #print(idx_second_neighbours[0])
    #print(idx_first_neighbours[0])
    #for i in idx_first_neighbours[0]:
    #    print(idx_first_neighbours[i])

    return idx_first_neighbours, idx_second_neighbours

def get_patch_dist_mat():
    xdist_array = np.reshape(patches[:,:,0],(N_particles*N_patches_i,1))
    ydist_array = np.reshape(patches[:,:,1],(N_particles*N_patches_i,1))
    xdist_array = xdist_array - xdist_array.transpose()
    ydist_array = ydist_array - ydist_array.transpose()
    return np.sqrt(np.power(xdist_array,2) + np.power(ydist_array,2))

def calc_Steinhardt(num_neighbours, box, pos):
    ql = freud.order.Steinhardt(num_neighbours,average=True)
    ql = ql.compute((box,pos), neighbors={"num_neighbors": num_neighbours}).particle_order
    mean_ql= np.mean(ql)
    return mean_ql

In [41]:
from matplotlib import cm
from matplotlib.collections import PatchCollection
from matplotlib.colorbar import Colorbar
from matplotlib.patches import Polygon, Circle
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable

def voronoi_plot(box, polytopes, idx_centers, idx_pores, idx_undefined, idx_selected, ax=None, DRAW_SELECTED = False):

    # Draw Voronoi polytopes
    patches_centers = [Polygon(poly[:, :2]) for idx,poly in enumerate(polytopes) if idx in idx_centers]
    patch_collection_centers = PatchCollection(patches_centers, edgecolors="black", facecolor='Red', alpha=0.4, linewidth=0.5)
    patches_pores = [Polygon(poly[:, :2]) for idx,poly in enumerate(polytopes) if idx in idx_pores]
    patch_collection_pores = PatchCollection(patches_pores, edgecolors="black", facecolor='Green', alpha=0.4, linewidth=0.5)
    patches_undefined = [Polygon(poly[:, :2]) for idx,poly in enumerate(polytopes) if idx in idx_undefined]
    patch_collection_undefined = PatchCollection(patches_undefined, edgecolors="black", facecolor='Gray',alpha=0.4, linewidth=0.5)
    if DRAW_SELECTED == True:
        patches_selected = [Polygon(poly[:, :2]) for idx,poly in enumerate(polytopes) if idx in idx_selected]
        patch_collection_selected = PatchCollection(patches_selected, edgecolors="black", facecolor='Black',alpha=1, linewidth=0.5)
        ax.add_collection(patch_collection_selected)

    ax.add_collection(patch_collection_centers)
    ax.add_collection(patch_collection_pores)
    ax.add_collection(patch_collection_undefined)

    # Draw box
    corners = [[0, 0, 0], [0, 1, 0], [1, 1, 0], [1, 0, 0]]
    # Need to copy the last point so that the box is closed.
    corners.append(corners[0])
    corners = box.make_absolute(corners)[:, :2]
    ax.plot(corners[:, 0], corners[:, 1], color="k", alpha=0.2)

    # Set title, limits, aspect
    ax.set_title("Voronoi Diagram - Cell Volume Analysis")
    ax.set_aspect("equal", "datalim")

    #add colorbar
    #ax_divider = make_axes_locatable(ax)
    #cax = ax_divider.append_axes("right", size="7%", pad="10%")
    #cb = Colorbar(cax, patch_collection)
    #cb.set_label("Polytope size")
    #cb.set_ticks([0,volume_cutoff, 1])
    #cb.set_ticklabels([f"<{volume_cutoff}", volume_cutoff, f">{volume_cutoff}"])
    return ax

def draw_polygons(vertices, ax=None):
    polygons = [Polygon(vert[:,:2]) for vert in vertices]
    polyong_collection = PatchCollection(polygons, edgecolors="black", facecolor='Blue', alpha=0.7, linewidth=0.01)
    ax.add_collection(polyong_collection)
    return ax


def draw_patches(patch_pos, ax=None):
    patches = [Circle(patch, radius=0.05) for patch in np.reshape(patch_pos, (-1,3))]
    patches_collection = PatchCollection(patches, edgecolors="black", facecolor='black', alpha=0.7, linewidth=0.01)
    ax.add_collection(patches_collection)
    return ax

def draw_voroplot(filename = None, SAVE = True, DPI=1000, DRAW_POLYGONS = True, DRAW_PATCHES = True,  DRAW_SELECTED = False, DRAW_NX=False):
    plt.figure(figsize=(15,15))
    ax = plt.gca()
    if DRAW_POLYGONS:
        vertices_voro = vertices - box_length/2
        draw_polygons(vertices_voro,ax)
    if DRAW_PATCHES:
        patches_voro  = patches - box_length/2
        draw_patches(patches_voro,ax)
    voronoi_plot(freud_box, voro.polytopes, idx_centers, idx_pores, idx_undefined, idx_selected, ax=ax, DRAW_SELECTED=DRAW_SELECTED)
    if DRAW_NX:
        nx.draw(G,pos=posdict,with_labels=False, font_weight='bold',ax = ax, node_size=patch_r)
    if SAVE:
        plt.savefig(filename, dpi=DPI)
    plt.show()
    return None



In [42]:
path = "../../../runs_varT_test/3asym/*/"
#path = "runs/mouse/*/"
dirlist = glob.glob(path)
for dir in dirlist:
    param = get_param_from_dir(dir)
    if param[1] != ['0.35', '0.05', '0.2', '0.25', '0.145', '1', '1']: #d r mu phi T eps id
        continue
    print(dir)
    patch_r = float(param[1][1])
    N_patches_i = get_number_of_patches(param[0])
    checkpoints = get_checkpoints(dir)

    for icp, cp in enumerate(checkpoints[1:2]):

        N_particles, positions, orient, box_xy = get_state()
        box_length = box_xy[0]

        vertices, patches = get_vertices_patches()

        #######calculate voronoi
        freud_box = freud.box.Box.square(box_length)
        vplot_points = collect_and_transform_points_for_voronoi()
        voro = freud.locality.Voronoi()
        voro.compute((freud_box, vplot_points))
        
        volume_cutoff = 0.44
        idx_centers = np.arange(0,N_particles)
        idx_pores = [idx for idx, vol in enumerate(voro.volumes) if (vol <= volume_cutoff) and (idx not in idx_centers)]
        idx_undefined = [idx for idx in range(N_particles,len(vplot_points)) if ((idx not in idx_centers) and (idx not in idx_pores))]
        #draw_voroplot(filename="../../../python_voronoi_plot.jpeg", SAVE = False, DRAW_POLYGONS = True, DRAW_PATCHES = True, DRAW_SELECTED=False)



        ######calculate clusters
        #calculate patch distances of neighbour particles

        idx_first_neighbours, idx_second_neighbours = get_voronoi_neighbours(voro.nlist)
        #idx_selected = idx_second_neighbours[10]
        #draw_voroplot(filename="../../../python_voronoi_plot.jpeg", SAVE = False, DRAW_POLYGONS = True, DRAW_PATCHES = True, DRAW_SELECTED=False)

        patches_distances = get_patch_dist_mat()
        patchrange = range(N_patches_i) 
        counter = 0
        idx_bonded = []
        G = nx.Graph()
        for i,neighs in enumerate(idx_second_neighbours[:N_particles]):
            for j in neighs:
                if j >= N_particles:
                    break
                for p1 in patchrange:
                    for p2 in patchrange:
                        if patches_distances[i*N_patches_i+p1,j*N_patches_i+p2] < 2 * patch_r:
                            counter+=1
                            idx_bonded.append(i)
                            G.add_edge(i,j)

        posdict = {}
        for i,pos in enumerate(vplot_points):
            posdict[i]=pos[:2]
        clusters = nx.connected_components(G)
        cluster_list = list(clusters)
        #print(idx_second_neighbours[70])
        #print(cluster_list)
        #plt.figure(figsize=(15,15))
        #ax = plt.gca()                    
        #nx.draw(G,pos=posdict,with_labels=True, font_weight='bold')
        #print(G.number_of_nodes())

        idx_selected=idx_bonded
        draw_voroplot(filename="../../../python_voronoi_plot.jpeg", SAVE = True, DRAW_POLYGONS = True, DRAW_PATCHES = True, DRAW_SELECTED=True, DRAW_NX=True)





../../../runs_varT_test/3asym/3asym_d0.35_r0.05_mu0.2_phi0.25_T0.145_eps-1_id1/


[[21.41393127 21.54021917]
 [ 9.55548861 18.93451668]
 [18.87272859 12.62056214]
 ...
 [14.94666177 -3.75592709]
 [15.74239655 -4.36157234]
 [15.86903333 -3.36962319]]


In [28]:


print(posdict)
#G = nx.Graph()
#G.add_edges_from(neighbours)
#plt.figure(figsize=(100,100))
#nx.draw(G,pos=posdict,with_labels=False, font_weight='bold')

#print(G.nodes)

{0: array([21.41393127, 21.54021917]), 1: array([ 9.55548861, 18.93451668]), 2: array([18.87272859, 12.62056214]), 3: array([ 4.42169412, 16.50270671]), 4: array([-18.99079736, -14.49179685]), 5: array([-18.00251289,  14.11656265]), 6: array([22.03947306, 10.77490191]), 7: array([22.2573921, -4.7744068]), 8: array([ 16.2685179 , -13.28251512]), 9: array([  2.68079221, -15.95075684]), 10: array([-17.06157205,  19.28937754]), 11: array([-12.98743835, -11.60959172]), 12: array([ 20.1512326 , -19.81586141]), 13: array([-21.48040437, -12.69697397]), 14: array([ 5.1455881 , -7.54573138]), 15: array([18.42428906, 14.68029053]), 16: array([ 7.50712428, -6.36632145]), 17: array([-16.73261007,  16.16687848]), 18: array([ 6.02207776, -4.8737542 ]), 19: array([  8.7465093 , -18.86210536]), 20: array([  1.11680791, -19.27742067]), 21: array([-15.11913116,  -3.15078669]), 22: array([ 5.91085184, 10.36157234]), 23: array([-7.13306639,  0.75776641]), 24: array([19.82842102, -0.67118427]), 25: array([ 